In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from main import *
import random
from library.hill_climbing.hc import hill_climb
from library.simulated_annealing.sa import simulated_annealing
from library.genetic_algorithm import gao

In [2]:
grid = np.zeros((10,10))
pieces = pieces_generator((10,10))
grid, pieces_coordinates = tetrimino_fitter(pieces, (10,10))

In [3]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        yaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True,),
        xaxis=dict(range=[-0.5, grid.shape[0] - 0.5], tickfont_size=12,),
    ),
)
fig.write_html(f"graphs/test_{grid.size}_size.html")
fig.show()


In [6]:
pop_size = 10
grid_shape = (10,10)
pieces_list = pieces_generator(grid_shape, rotation=False)
# total_pieces = [letter for (letter, number) in total_pieces]
individuals = [generate_individual(pieces_list) for i in range(pop_size)]
pop = Population(individuals, "max", 3, pieces_list, grid_shape)

In [8]:
gao.GAO(pop, 5, tournament_size=4, p_crossover=0.01, p_mutation=0.5, n_mutations=1, p_adoption=0.01)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        title=f"Fitness: {pop.elites[0].fitness}",
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()

ELITES: [Individual <88.0>, Individual <84.0>, Individual <84.0>]
ALL: [Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <76.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>]
elite @ prime: [Individual <84.0>, Individual <84.0>, Individual <84.0>]
[Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <80.0>, Individual <80.0>, Individual <76.0>, Individual <76.0>, Individual <76.0>]


ELITES: [Individual <84.0>, Individual <84.0>, Individual <84.0>]
ALL: [Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <80.0>, Individual <76.0>, Individual <80.0>, Individual <80.0>, Individual <76.0>, Individual <84.0>, Individual <76.0>]
elite @ prime: [Individual <84.0>, Individual <84.0>, Individual <84.0>]
[Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <84.0>, Individual <80.0>, Individual <80.0>, Ind

In [7]:
px.line(pop.fitness_history)

In [13]:
# Hill Climbing
pop = Population(individuals, "max", 1, pieces_list, grid_shape)
hill_climb(pop, hardstop=1)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        title=f"Fitness: {pop.elites[0].fitness}",
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()

In [15]:
# Simulated Annealing
pop = Population(individuals, "max", 1, pieces_list, grid_shape)
simulated_annealing(pop)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        title=f"Fitness: {pop.elites[0].fitness}",
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()

KeyboardInterrupt: 